In [135]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

import pandas as pd
import re
import nltk

In [114]:
data = pd.read_csv("Tweets.csv")

In [115]:
data = data[data['airline_sentiment'] != 'neutral']

In [117]:
data2 = data[["airline_sentiment", "airline", "text"]]

In [118]:
airnames = data2["airline"].unique()
airnames = pd.Series(airnames)
airnames = airnames.str.lower()

airnames2 = airnames.str.replace(" ", "")
airnames = airnames.append(airnames2)
others = pd.Series(['americanair', '@'])
airnames = others.append(airnames)
airnames

0       americanair
1                 @
0    virgin america
1            united
2         southwest
3             delta
4        us airways
5          american
0     virginamerica
1            united
2         southwest
3             delta
4         usairways
5          american
dtype: object

In [119]:
proc_text = data2['text'].str.lower()

for i in airnames:
    proc_text = proc_text.str.replace(i, '')

In [120]:
data3 = pd.concat([proc_text, data["airline"], data["airline_sentiment"]], axis=1)

In [121]:
data3.head()

,text,airline,airline_sentiment
1,plus you've added commercials to the experien...,Virgin America,positive
3,"it's really aggressive to blast obnoxious ""en...",Virgin America,negative
4,and it's a really big bad thing about it,Virgin America,negative
5,seriously would pay $30 a flight for seats th...,Virgin America,negative
6,"yes, nearly every time i fly vx this “ear wor...",Virgin America,positive


In [127]:
vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = 1,
    max_features = 1000
)

features = vectorizer.fit_transform(
    proc_text
)

features_nd = features.toarray()

In [162]:
#print(vectorizer.get_feature_names())
#features_nd.shape
features_nd

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=int64)

In [149]:
dataVA = data3[data3['airline'] == 'Virgin America']
dataU = data3[data3['airline'] == 'United']
dataSW = data3[data3['airline'] == 'Southwest']
dataD = data3[data3['airline'] == 'Delta']
dataUSA = data3[data3['airline'] == 'US Airways']
dataA = data3[data3['airline'] == 'American']

In [150]:
train_dataVA = dataVA[:int(len(dataVA)*.8)]
train_dataU = dataU[:int(len(dataU)*.8)]
train_dataSW = dataSW[:int(len(dataSW)*.8)]
train_dataD = dataD[:int(len(dataD)*.8)]
train_dataUSA = dataUSA[:int(len(dataUSA)*.8)]
train_dataA = dataA[:int(len(dataA)*.8)]

train_data = pd.concat([train_dataVA,
                      train_dataU,
                      train_dataSW,
                      train_dataD,
                      train_dataUSA,
                      train_dataA], axis = 0)
train_data

,text,airline,airline_sentiment
1,plus you've added commercials to the experien...,Virgin America,positive
3,"it's really aggressive to blast obnoxious ""en...",Virgin America,negative
4,and it's a really big bad thing about it,Virgin America,negative
5,seriously would pay $30 a flight for seats th...,Virgin America,negative
6,"yes, nearly every time i fly vx this “ear wor...",Virgin America,positive
8,"well, i didn't…but now i do! :-d",Virgin America,positive
9,"it was amazing, and arrived an hour early. yo...",Virgin America,positive
11,i &lt;3 pretty graphics. so much better than ...,Virgin America,positive
12,this is such a great deal! already thinking a...,Virgin America,positive
13,virginmedia i'm flying your #fabulous #seduct...,Virgin America,positive


In [151]:
test_dataVA = dataVA[int(len(dataVA)*.8):]
test_dataU = dataU[int(len(dataU)*.8):]
test_dataSW = dataSW[int(len(dataSW)*.8):]
test_dataD = dataD[int(len(dataD)*.8):]
test_dataUSA = dataUSA[int(len(dataUSA)*.8):]
test_dataA = dataA[int(len(dataA)*.8):]

test_data = pd.concat([test_dataVA,
                      test_dataU,
                      test_dataSW,
                      test_dataD,
                      test_dataUSA,
                      test_dataA], axis = 0)
test_data

,text,airline,airline_sentiment
397,good to be home #texas #moodlighting http://t...,Virgin America,positive
398,i cannot even open my laptop in seat 4c and i...,Virgin America,negative
399,i am in seat 4c and i cannot even open my lap...,Virgin America,negative
401,thanks for that. been needing a way to make t...,Virgin America,positive
404,this is too cool! never been on ur planes bu...,Virgin America,positive
406,great. well deserved.,Virgin America,positive
407,fyi the info.com email address you say to con...,Virgin America,negative
408,status match - 2 weeks have gone by and no ne...,Virgin America,negative
409,your website sucks donkey dicks. just thought...,Virgin America,negative
411,all crap channels which is why i pay to watch...,Virgin America,negative


In [152]:
features_VA = features_nd[:len(dataVA)]
features_U = features_nd[len(dataVA): len(dataVA)+len(dataU)]
features_SW = features_nd[len(dataVA)+len(dataU): len(dataVA)+len(dataU)+len(dataSW)]
features_D = features_nd[len(dataVA)+len(dataU)+len(dataSW): len(dataVA)+len(dataU)+len(dataSW)+len(dataD)]
features_USA = features_nd[len(dataVA)+len(dataU)+len(dataSW)+len(dataD): len(dataVA)+len(dataU)+len(dataSW)+len(dataD)+len(dataUSA)]
features_A = features_nd[len(dataVA)+len(dataU)+len(dataSW)+len(dataD)+len(dataUSA): len(dataVA)+len(dataU)+len(dataSW)+len(dataD)+len(dataUSA)+len(dataA)]

In [153]:
train_featuresVA = pd.DataFrame(features_VA[:int(len(features_VA)*.8)])
train_featuresU = pd.DataFrame(features_U[:int(len(features_U)*.8)])
train_featuresSW = pd.DataFrame(features_SW[:int(len(features_SW)*.8)])
train_featuresD = pd.DataFrame(features_D[:int(len(features_D)*.8)])
train_featuresUSA = pd.DataFrame(features_USA[:int(len(features_USA)*.8)])
train_featuresA = pd.DataFrame(features_A[:int(len(features_A)*.8)])

train_features = pd.concat([train_featuresVA,
                      train_featuresU,
                      train_featuresSW,
                      train_featuresD,
                      train_featuresUSA,
                      train_featuresA], axis = 0)
train_features = train_features.to_numpy()
train_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [154]:
test_featuresVA = pd.DataFrame(features_VA[int(len(features_VA)*.8):])
test_featuresU = pd.DataFrame(features_U[int(len(features_U)*.8):])
test_featuresSW = pd.DataFrame(features_SW[int(len(features_SW)*.8):])
test_featuresD = pd.DataFrame(features_D[int(len(features_D)*.8):])
test_featuresUSA = pd.DataFrame(features_USA[int(len(features_USA)*.8):])
test_featuresA = pd.DataFrame(features_A[int(len(features_A)*.8):])

test_features = pd.concat([test_featuresVA,
                      test_featuresU,
                      test_featuresSW,
                      test_featuresD,
                      test_featuresUSA,
                      test_featuresA], axis = 0)
test_features = test_features.to_numpy()
test_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]], dtype=int64)

In [156]:
log_model = LogisticRegression()
log_model = log_model.fit(X=train_features, y=train_data['airline_sentiment'])
y_pred = log_model.predict(test_features)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [159]:
from sklearn.metrics import accuracy_score

y_test = test_data['airline_sentiment']
print(accuracy_score(y_test, y_pred))

0.8641280830809174
